In [2]:
import mkwikidata, pprint
import pandas as pd

In [9]:
# first query
query = """
SELECT ?child ?childLabel
WHERE
{
    # child has parent 'Bach'
    ?child wdt:P22 wd:Q1339;
           wdt:P25 wd:Q57487;
           wdt:P106 wd:Q36834,
                   wd:Q486748.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
qr = mkwikidata.run_query(query)
qr_df = pd.json_normalize(qr['results']['bindings'])
qr_df.head()

,child.type,child.value,childLabel.xml:lang,childLabel.type,childLabel.value
0,uri,http://www.wikidata.org/entity/Q107277,en,literal,Wilhelm Friedemann Bach


Common pattern:

```
SELECT ?s1 ?s2 ?s3
WHERE
{
  ?s1 p1 o1;
      p2 o2;
      p3 o31, o32, o33.
  ?s2 p4 o41, o42.
  ?s3 p5 o5;
      p6 o6.
}
```

In [3]:
# second query
query = """
SELECT ?label ?placelabel ?coord  # three variables
WHERE {
    # here we set ?type as a group of items
    VALUES ?type {wd:Q571 wd:Q7725634}  # book or literary work
    ?item wdt:P31 ?type.  # item is of type ?type - instance of ?type
    ?item rdfs:label ?label FILTER(lang(?label) = "en").  # item has label ?label
    ?item wdt:P840 ?place.  # item has narrative location ?place
    ?place wdt:P31/wdt:P279* wd:Q515. # ?place is a subclass of administrative territorial entity
    ?place wdt:P625 ?coord .  # has coordinate location ?coord
    ?place rdfs:label ?placelabel FILTER(lang(?placelabel) = "en").
}
"""

query_result = mkwikidata.run_query(query)

In [4]:
qr_df2 = pd.json_normalize(query_result['results']['bindings'])
qr_df2.head()

,coord.datatype,coord.type,coord.value,label.xml:lang,label.type,label.value,placelabel.xml:lang,placelabel.type,placelabel.value
0,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(-122.416388888 37.7775),en,literal,Zodiac,en,literal,San Francisco
1,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(-122.416388888 37.7775),en,literal,American Gods,en,literal,San Francisco
2,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(11.254166666 43.771388888),en,literal,The Decameron,en,literal,Florence
3,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(11.254166666 43.771388888),en,literal,A Room with a View,en,literal,Florence
4,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(11.254166666 43.771388888),en,literal,Day 8 Tale 5 of the Decameron,en,literal,Florence


In [5]:
qr_df2[['label.value', 'placelabel.value', 'coord.value']].head()

,label.value,placelabel.value,coord.value
0,Zodiac,San Francisco,Point(-122.416388888 37.7775)
1,American Gods,San Francisco,Point(-122.416388888 37.7775)
2,The Decameron,Florence,Point(11.254166666 43.771388888)
3,A Room with a View,Florence,Point(11.254166666 43.771388888)
4,Day 8 Tale 5 of the Decameron,Florence,Point(11.254166666 43.771388888)


This shows that SPARQL is a powerful language to retrieve data from wikidata, such as
the coordinates of a city, the population of a country, the date of birth of a person, etc. Sometimes, one has to pay for the data if you are trying to get coordinates of a city from other sources. But with SPARQL, you can get it for free.

In [6]:
qr_df2.shape

(13052, 9)

In [7]:
# Query 3: What is the most common narration country in Wikidata books and literature works?

query = """
SELECT ?countryLabel (COUNT(?country) AS ?count)
WHERE {
    VALUES ?type {wd:Q571 wd:Q7725634}  # book or literary work
    ?item wdt:P31 ?type.  # item is of type ?type - instance of ?type
    ?item wdt:P840 ?place.  # item has narrative location ?place
    ?place wdt:P17 ?country .  # ?place is in country ?country
    ?country rdfs:label ?countryLabel FILTER(lang(?countryLabel) = "en").
}
GROUP BY ?countryLabel
ORDER BY DESC(?count)
"""
qr3 = mkwikidata.run_query(query)
qr_df3 = pd.json_normalize(qr3['results']['bindings'])

In [8]:
qr_df3.head()

,countryLabel.xml:lang,countryLabel.type,countryLabel.value,count.datatype,count.type,count.value
0,en,literal,United States of America,http://www.w3.org/2001/XMLSchema#integer,literal,2484
1,en,literal,United Kingdom,http://www.w3.org/2001/XMLSchema#integer,literal,1576
2,en,literal,France,http://www.w3.org/2001/XMLSchema#integer,literal,1336
3,en,literal,Italy,http://www.w3.org/2001/XMLSchema#integer,literal,730
4,en,literal,Spain,http://www.w3.org/2001/XMLSchema#integer,literal,601


In [10]:
qr_df3[['countryLabel.value', 'count.value']].head(10)

,countryLabel.value,count.value
0,United States of America,2484
1,United Kingdom,1576
2,France,1336
3,Italy,730
4,Spain,601
5,Denmark,540
6,Germany,365
7,Serbia,331
8,India,243
9,Sweden,211


In [1]:
# use sparqlwrapper
from SPARQLWrapper import SPARQLWrapper, JSON, XML, RDF

In [3]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql") #determine SPARQL endpoint
sparql.setReturnFormat(JSON) #determine the output format

In [4]:
# Let's see what is the most common cause of death among writers?
sparql.setQuery("""
SELECT ?cause ?causeLabel (COUNT(?person) AS ?count)
WHERE {
    ?person wdt:P106 wd:Q36180.  # person has occupation writer
    ?person wdt:P509 ?cause.  # person has cause of death ?cause
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?cause ?causeLabel
HAVING (?count > 1)
ORDER BY DESC(?count)
""")
results = sparql.query().convert()  # get the results
qr_df4 = pd.json_normalize(results['results']['bindings'])

In [5]:
qr_df4.shape

(287, 8)

In [6]:
qr_df4.head()

,cause.type,cause.value,causeLabel.xml:lang,causeLabel.type,causeLabel.value,count.datatype,count.type,count.value
0,uri,http://www.wikidata.org/entity/Q12078,en,literal,cancer,http://www.w3.org/2001/XMLSchema#integer,literal,1125
1,uri,http://www.wikidata.org/entity/Q12152,en,literal,myocardial infarction,http://www.w3.org/2001/XMLSchema#integer,literal,1076
2,uri,http://www.wikidata.org/entity/Q12204,en,literal,tuberculosis,http://www.w3.org/2001/XMLSchema#integer,literal,579
3,uri,http://www.wikidata.org/entity/Q12192,en,literal,pneumonia,http://www.w3.org/2001/XMLSchema#integer,literal,543
4,uri,http://www.wikidata.org/entity/Q12202,en,literal,stroke,http://www.w3.org/2001/XMLSchema#integer,literal,439


In [7]:
qr_df4[['causeLabel.value', 'count.value']].head(10)

,causeLabel.value,count.value
0,cancer,1125
1,myocardial infarction,1076
2,tuberculosis,579
3,pneumonia,543
4,stroke,439
5,disease,394
6,lung cancer,385
7,traffic collision,296
8,ballistic trauma,281
9,heart failure,241


In [8]:
# now let's check musicians wd:Q639669
sparql.setQuery("""
SELECT ?cause ?causeLabel (COUNT(?person) AS ?count)
WHERE {
    ?person wdt:P106 wd:Q639669.  # person has occupation musician
    ?person wdt:P509 ?cause.  # person has cause of death ?cause
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?cause ?causeLabel
HAVING (?count > 1)
ORDER BY DESC(?count)
""")

results = sparql.query().convert()  # get the results
qr_df5 = pd.json_normalize(results['results']['bindings'])

In [9]:
qr_df5.shape

(122, 8)

In [10]:
qr_df5[['causeLabel.value', 'count.value']].head(10)

,causeLabel.value,count.value
0,myocardial infarction,310
1,cancer,261
2,traffic collision,138
3,lung cancer,135
4,pneumonia,94
5,COVID-19,88
6,heart failure,84
7,disease,74
8,stroke,66
9,pancreatic cancer,48


In [12]:
#  writers that died before 1800
sparql.setQuery("""
SELECT ?cause ?causeLabel (COUNT(?person) AS ?count)
WHERE
{
  ?person wdt:P106 wd:Q36180; #ocupation writer
          wdt:P509 ?cause ; #?cause is a death cause
          wdt:P570 ?date . #death date
  FILTER (?date < "1800-01-01T00:00:00Z"^^xsd:dateTime) . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?cause ?causeLabel
HAVING(?count > 1)
ORDER BY DESC(?count)
""")

results = sparql.query().convert()  # get the results
qr_df6 = pd.json_normalize(results['results']['bindings'])

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'SPARQL-QUERY: queryStr=\nSELECT ?cause ?causeLabel (COUNT(?person) AS ?count)\nWHERE\n{\n  ?person wdt:P106 wd:Q36180; #ocupation writer\n          wdt:P509 ?cause ; #?cause is a death cause\n          wdt:P570 ?date . #death date\n  FILTER (?date < "1800-01-01T00:00:00Z"^^xsd:dateTime) . \n  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }\n}\nGROUP BY ?cause ?causeLabel\nHAVING(?count > 1)\nORDER BY DESC(?count)\n\njava.util.concurrent.TimeoutException\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:205)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:292)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doSparqlQuery(QueryServlet.java:678)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doGet(QueryServlet.java:290)\n\tat com.bigdata.rdf.sail.webapp.RESTServlet.doGet(RESTServlet.java:240)\n\tat com.bigdata.rdf.sail.web

In [ ]:
qr_df6[['causeLabel.value', 'count.value']].head(10)

In [13]:
# dbpedia
sparql = SPARQLWrapper("http://dbpedia.org/sparql") #determine SPARQL endpoint
sparql.setReturnFormat(JSON) #determine the output format

In [14]:
#SPARQL query to be executed
sparql.setQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbc: <http://dbpedia.org/resource/Category:>
PREFIX dct: <http://purl.org/dc/terms/>

Select distinct ?date ?author ?authorName ?thumbnail ?description 


WHERE {
?author rdf:type dbo:Writer ;
              dct:subject dbc:Nobel_laureates_in_Literature ;
        rdfs:label ?authorName ;
        dbo:wikiPageWikiLink ?link ;
        rdfs:comment ?description 
 FILTER ((lang(?authorName)="en")&&(lang(?description)="en")) .

?link dct:subject dbc:Nobel_Prize_in_Literature ;
         dbp:holderLabel ?date .
 OPTIONAL { ?author dbo:thumbnail ?thumbnail . }
} 

ORDER BY ?date

""")

sparql.setReturnFormat(JSON)   # Return format is JSON
results = sparql.query().convert()   # execute SPARQL query and write result to "results"

In [15]:
results

{'head': {'link': [],
  'vars': ['date', 'author', 'authorName', 'thumbnail', 'description']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'date': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Currently held by'},
    'author': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Camilo_José_Cela'},
    'authorName': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Camilo José Cela'},
    'thumbnail': {'type': 'uri',
     'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Camilo_José_Cela._Fototeca._Biblioteca_Virtual_del_Patrimonio_Bibliográfico.jpg?width=300'},
    'description': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Camilo José Cela y Trulock, 1st Marquess of Iria Flavia (Spanish: [kamilo xoˈse ˈθela]; 11 May 1916 – 17 January 2002) was a Spanish novelist, poet, story writer and essayist associated with the Generation of \'36 movement. He was awarded the 1989 Nobel Prize in Literature "for a rich and int

In [16]:
qr_df7 = pd.json_normalize(results['results']['bindings'])
qr_df7.head()

,date.type,date.xml:lang,date.value,author.type,author.value,authorName.type,authorName.xml:lang,authorName.value,thumbnail.type,thumbnail.value,description.type,description.xml:lang,description.value
0,literal,en,Currently held by,uri,http://dbpedia.org/resource/Camilo_José_Cela,literal,en,Camilo José Cela,uri,http://commons.wikimedia.org/wiki/Special:File...,literal,en,"Camilo José Cela y Trulock, 1st Marquess of Ir..."
1,literal,en,Currently held by,uri,http://dbpedia.org/resource/Carl_Spitteler,literal,en,Carl Spitteler,uri,http://commons.wikimedia.org/wiki/Special:File...,literal,en,Carl Friedrich Georg Spitteler (24 April 1845 ...
2,literal,en,Currently held by,uri,http://dbpedia.org/resource/Pär_Lagerkvist,literal,en,Pär Lagerkvist,uri,http://commons.wikimedia.org/wiki/Special:File...,literal,en,Pär Fabian Lagerkvist (23 May 1891 – 11 July 1...
3,literal,en,Currently held by,uri,http://dbpedia.org/resource/Roger_Martin_du_Gard,literal,en,Roger Martin du Gard,uri,http://commons.wikimedia.org/wiki/Special:File...,literal,en,Roger Martin du Gard (French: [dy gaʁ]; 23 Mar...
4,literal,en,Currently held by,uri,http://dbpedia.org/resource/Romain_Rolland,literal,en,Romain Rolland,uri,http://commons.wikimedia.org/wiki/Special:File...,literal,en,Romain Rolland (French: [ʁɔlɑ̃]; 29 January 18...


In [17]:
qr_df7[['date.value', 'authorName.value', 'thumbnail.value', 'description.value']].head(10)

,date.value,authorName.value,thumbnail.value,description.value
0,Currently held by,Camilo José Cela,http://commons.wikimedia.org/wiki/Special:File...,"Camilo José Cela y Trulock, 1st Marquess of Ir..."
1,Currently held by,Carl Spitteler,http://commons.wikimedia.org/wiki/Special:File...,Carl Friedrich Georg Spitteler (24 April 1845 ...
2,Currently held by,Pär Lagerkvist,http://commons.wikimedia.org/wiki/Special:File...,Pär Fabian Lagerkvist (23 May 1891 – 11 July 1...
3,Currently held by,Roger Martin du Gard,http://commons.wikimedia.org/wiki/Special:File...,Roger Martin du Gard (French: [dy gaʁ]; 23 Mar...
4,Currently held by,Romain Rolland,http://commons.wikimedia.org/wiki/Special:File...,Romain Rolland (French: [ʁɔlɑ̃]; 29 January 18...
5,Currently held by,Rudyard Kipling,http://commons.wikimedia.org/wiki/Special:File...,Joseph Rudyard Kipling (/ˈrʌdjərd/ RUD-yərd; 3...
6,Currently held by,Samuel Beckett,http://commons.wikimedia.org/wiki/Special:File...,Samuel Barclay Beckett (/ˈbɛkɪt/; 13 April 190...
7,Currently held by,Saul Bellow,http://commons.wikimedia.org/wiki/Special:File...,Saul Bellow (born Solomon Bellows; 10 July 191...
8,Currently held by,Bjørnstjerne Bjørnson,http://commons.wikimedia.org/wiki/Special:File...,Bjørnstjerne Martinius Bjørnson (/ˈbjɜːrnsən/ ...
9,Currently held by,Boris Pasternak,http://commons.wikimedia.org/wiki/Special:File...,Boris Leonidovich Pasternak (/ˈpæstərnæk/; Rus...
